# Planejamento da Solução

**Qual é o problema de negócio?**

- Empresa alimentícia situada no RJ, deseja abrir filiais na cidade de São Paulo. Para isso, precisa de 3 análises:
    - 1 - Estimar faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão, neste caso sem séries temporais).
    - 2 - Classificar o potencial dos bairros de São Paulo em alto, médio ou baixo (classificação multiclasse).
    - 3 - Segmentar os bairros de São Paulo de acordo com o perfil de renda e idade, identificando os com maior aderência ao público alvo.
    
- Público alvo: adultos de 25 a 50 anos, das classes A (rendas A1 e A2) e B (rendas B1 e B2). 

**Saída**

- Prototipagem técnica da solução: https://docs.google.com/spreadsheets/d/17lxCBRLPEuNCO25BimVFRE3Tms-314WSTpJ1xgEIf38/edit#gid=0

- O que será entregue, efetivamente? / Onde o time de negócio quer ver?
    - Documento no formato doc, pdf ou ppt, voltado ao negócio, apresentando um racional de como os dados foram analisados. Detalhar com gráficos, tabelas, e descrever conclusões.
    - Responder: Dada a natureza do problema apresentado, que outro dado externo (fontes públicas ou privadas) poderia ser utilizado para agregar mais valor ao resultado? Por que?
    
**Entrada**
- Fontes de dados:
    - Dataset contendo faturamento e potencial dos bairros do Rio de Janeiro do cliente, bem como dados sociodemográficos do bairros do Rio de Janeiro e São Paulo.

- Ferramentas:
    - Python 3.8.12, Jupyter Notebook, Git, Github.

**Processamento**
- Tipo de problema: Análise exploratória, regressão, classificação e clusterização.


**Estratégia de condução**
- Ciclo 1: 
    - Entendimento do problema de negócio e planejamento da solução, coleta de dados, descrição dos dados, split do dataset entre treino, validação e teste, criação de modelos de machine learning para análises 1, 2 e 3.
- Ciclo 2: 
    - Inclusão de limpeza de dados, EDA (análise exploratória dos dados), modelagem de dados, e retreino dos modelos das análises 1, 2 e 3.

# Importações

## Bibliotecas

In [233]:
import pandas                      as pd 
import seaborn                     as sns
import numpy                       as np
from IPython.core.display          import HTML
from matplotlib                    import pyplot as plt
from tabulate                      import tabulate
import inflection
#sklearn
from sklearn.model_selection       import train_test_split
from sklearn.neighbors             import KNeighborsRegressor
from sklearn.neighbors             import KNeighborsClassifier

## Funções Auxiliares

In [3]:
def jupyter_settings():
    """ Otimiza configurações gerais, padronizanod tamanhos de plots, etc """
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)
jupyter_settings()

## Coleta de Dados

Carregamento dos dados originais do xlsx.

In [5]:
df_raw = pd.read_excel("../data/data_raw.xlsx")
df_raw

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial
0,3304557060,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501,932515.0,Médio
1,3304557138,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,588833.0,Baixo
2,3304557057,Água Santa,Rio de Janeiro,RJ,9003,883,399,597,762,1755,2076,1112,1419,0,96,404,652,644,522,77,254,2391,874200.0,Baixo
3,3304557031,Alto Da Boa Vista,Rio de Janeiro,RJ,9606,1072,538,660,685,1317,2007,1341,1986,114,178,393,517,945,584,137,286,3727,912226.0,Médio
4,3304557125,Anchieta,Rio de Janeiro,RJ,57222,7677,3774,4892,4600,8660,12272,7157,8190,0,0,1089,2821,5110,5422,1073,3261,1380,553020.0,Médio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,355030213,Vila Sônia,São Paulo,SP,34061,2908,1462,2253,2076,4579,7557,5227,7999,942,1089,2866,2219,2216,1227,306,1098,5285,NaN,NaN
452,355030207,Vila Suzana,São Paulo,SP,35403,4127,1890,2678,2433,5855,9107,4608,4705,1726,1522,2458,1186,1166,918,209,3840,7418,NaN,NaN
453,355030162,Vila Terezinha,São Paulo,SP,122359,18304,9304,13258,9965,19248,26592,12579,13109,0,0,1758,4517,9450,11473,3218,7540,1252,NaN,NaN
454,355030157,Vila Zatt,São Paulo,SP,125864,14670,7305,11225,9338,18841,28500,16700,19285,8,872,5093,8063,10012,8082,2856,6853,1936,NaN,NaN


# Limpeza e Análise Preliminar

## Renomear Colunas

Padroinzar nome das variáveis no formato snake_case, melhorando sua legibilidade.

In [6]:
df_raw.columns

Index(['codigo', 'nome', 'cidade', 'estado', 'população', 'popAte9',
       'popDe10a14', 'popDe15a19', 'popDe20a24', 'popDe25a34', 'popDe35a49',
       'popDe50a59', 'popMaisDe60', 'domiciliosA1', 'domiciliosA2',
       'domiciliosB1', 'domiciliosB2', 'domiciliosC1', 'domiciliosC2',
       'domiciliosD', 'domiciliosE', 'rendaMedia', 'faturamento', 'potencial'],
      dtype='object')

In [7]:
cols_old = ['codigo', 'nome', 'cidade', 'estado', 'populacao', 'popAte9',
       'popDe10a14', 'popDe15a19', 'popDe20a24', 'popDe25a34', 'popDe35a49',
       'popDe50a59', 'popMaisDe60', 'domiciliosA1', 'domiciliosA2',
       'domiciliosB1', 'domiciliosB2', 'domiciliosC1', 'domiciliosC2',
       'domiciliosD', 'domiciliosE', 'rendaMedia', 'faturamento', 'potencial']
snakecase = lambda x: inflection.underscore ( x )
cols_new = list( map( snakecase, cols_old ) )
df_raw.columns = cols_new
df_raw.columns

Index(['codigo', 'nome', 'cidade', 'estado', 'populacao', 'pop_ate9',
       'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34',
       'pop_de35a49', 'pop_de50a59', 'pop_mais_de60', 'domicilios_a1',
       'domicilios_a2', 'domicilios_b1', 'domicilios_b2', 'domicilios_c1',
       'domicilios_c2', 'domicilios_d', 'domicilios_e', 'renda_media',
       'faturamento', 'potencial'],
      dtype='object')

## Dicionário de Dados

Compreender com clareza o significado de cada variável é fundamental para avançar no projeto.

In [8]:
tab = [['Coluna', 'Significado'],   
       ['codigo', 'Código do bairro'],
       ['nome', 'Nome do bairro'],
       ['cidade', 'Cidade'],
       ['estado', 'Estado'],
       ['populacao', 'População total'],
       ['pop_ate9', 'População - até 9 anos'],
       ['pop_de10a14', 'População - de 10 a 14 anos'],
       ['pop_de15a19', 'População - de 15 a 19 anos'],
       ['pop_de20a24', 'População - de 20 a 24 anos'],
       ['pop_de25a34', 'População - de 25 a 34 anos'],
       ['pop_de35a49', 'População - de 35 a 49 anos'],
       ['pop_de50a59', 'População - de 50 a 59 anos'],
       ['pop_mais_de60', 'População - 60 anos ou mais'],
       ['domicilios_a1', 'Quantidade de Domicílios de Renda A1'],
       ['domicilios_a2', 'Quantidade de Domicílios de Renda A2'],
       ['domicilios_b1', 'Quantidade de Domicílios de Renda B1'],
       ['domicilios_b2', 'Quantidade de Domicílios de Renda B2'],
       ['domicilios_c1', 'Quantidade de Domicílios de Renda C1'],
       ['domicilios_c2', 'Quantidade de Domicílios de Renda C2'],
       ['domicilios_d', 'Quantidade de Domicílios de Renda D'],
       ['domicilios_e', 'Quantidade de Domicílios de Renda E'],
       ['renda_media', 'Renda Média por Domicílio'],
       ['faturamento', 'Faturamento Total no Bairro'],
       ['potencial', 'Potencial do Bairro']]
print(tabulate(tab, headers='firstrow', stralign='left')) #tablefmt='pipe', tablefmt='grid'

Coluna         Significado
-------------  ------------------------------------
codigo         Código do bairro
nome           Nome do bairro
cidade         Cidade
estado         Estado
populacao      População total
pop_ate9       População - até 9 anos
pop_de10a14    População - de 10 a 14 anos
pop_de15a19    População - de 15 a 19 anos
pop_de20a24    População - de 20 a 24 anos
pop_de25a34    População - de 25 a 34 anos
pop_de35a49    População - de 35 a 49 anos
pop_de50a59    População - de 50 a 59 anos
pop_mais_de60  População - 60 anos ou mais
domicilios_a1  Quantidade de Domicílios de Renda A1
domicilios_a2  Quantidade de Domicílios de Renda A2
domicilios_b1  Quantidade de Domicílios de Renda B1
domicilios_b2  Quantidade de Domicílios de Renda B2
domicilios_c1  Quantidade de Domicílios de Renda C1
domicilios_c2  Quantidade de Domicílios de Renda C2
domicilios_d   Quantidade de Domicílios de Renda D
domicilios_e   Quantidade de Domicílios de Renda E
renda_media    Renda Média por 

## Tipos de Dados

Avaliar tipos de dados, e necessidade de conversões.

In [9]:
df_raw.head(2)

,codigo,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
0,3304557060,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501,932515.0,Médio
1,3304557138,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,588833.0,Baixo


In [10]:
df_raw.dtypes

codigo             int64
nome              object
cidade            object
estado            object
populacao          int64
pop_ate9           int64
pop_de10a14        int64
pop_de15a19        int64
pop_de20a24        int64
pop_de25a34        int64
pop_de35a49        int64
pop_de50a59        int64
pop_mais_de60      int64
domicilios_a1      int64
domicilios_a2      int64
domicilios_b1      int64
domicilios_b2      int64
domicilios_c1      int64
domicilios_c2      int64
domicilios_d       int64
domicilios_e       int64
renda_media       object
faturamento      float64
potencial         object
dtype: object

No momento, não será possível converter "faturamento" para int (visto que não utiliza as casas decimais), em função dos dados de SP, onde há registros nulos. Farei isto após a separação dos datasets.

Demais variáveis estão ok.

## Conferência de Nulos

Avaliação de tratativa de registros nulos.

In [12]:
df_raw.isna().sum()

codigo             0
nome               0
cidade             0
estado             0
populacao          0
pop_ate9           0
pop_de10a14        0
pop_de15a19        0
pop_de20a24        0
pop_de25a34        0
pop_de35a49        0
pop_de50a59        0
pop_mais_de60      0
domicilios_a1      0
domicilios_a2      0
domicilios_b1      0
domicilios_b2      0
domicilios_c1      0
domicilios_c2      0
domicilios_d       0
domicilios_e       0
renda_media        6
faturamento      296
potencial        296
dtype: int64

Conferir se os registros com valores nulos em "faturamento" e "potencial" são apenas os registros de SP.

In [56]:
df_raw.loc[(df_raw.potencial.isna() == True) & (df_raw.faturamento.isna() == True) & (df_raw.estado == "SP")]

,codigo,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,NaN,NaN
161,35503020,Aclimação,São Paulo,SP,32791,2297,1017,2096,2197,5341,7281,4917,7645,1413,1734,3704,2351,1946,827,291,1617,5920,NaN,NaN
162,355030285,Adventista,São Paulo,SP,104193,15070,7343,10631,8657,17749,23364,11567,9812,0,0,1423,4875,8595,10082,3111,5776,1284,NaN,NaN
163,35503088,Água Branca,São Paulo,SP,12721,953,343,627,819,2142,2833,1790,3214,624,667,1558,1032,915,361,84,404,6278,NaN,NaN
164,35503066,Água Funda,São Paulo,SP,48417,5078,2396,4018,3571,7388,10751,6648,8567,0,303,1794,2986,4489,2836,1104,2553,1905,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,355030213,Vila Sônia,São Paulo,SP,34061,2908,1462,2253,2076,4579,7557,5227,7999,942,1089,2866,2219,2216,1227,306,1098,5285,NaN,NaN
452,355030207,Vila Suzana,São Paulo,SP,35403,4127,1890,2678,2433,5855,9107,4608,4705,1726,1522,2458,1186,1166,918,209,3840,7418,NaN,NaN
453,355030162,Vila Terezinha,São Paulo,SP,122359,18304,9304,13258,9965,19248,26592,12579,13109,0,0,1758,4517,9450,11473,3218,7540,1252,NaN,NaN
454,355030157,Vila Zatt,São Paulo,SP,125864,14670,7305,11225,9338,18841,28500,16700,19285,8,872,5093,8063,10012,8082,2856,6853,1936,NaN,NaN


Confirmado, "faturametno" será tratado abaixo. Demais variáveis sem necessidade de tratativas.

## Separação entre RJ e SP

Separação do dataset em duas pares, sendo:
- df_rj: registros onde estado = RJ, contendo valores nas variáveis alvo "faturamento" e "potencial".
- df_sp: registros onde estado = SP, contendo valores nulos para as variáveis alvo.

In [106]:
df_rj = df_raw.loc[df_raw["estado"] == "RJ"]
df_sp = df_raw.loc[df_raw["estado"] == "SP"]

print (f"Número de linhas e colunas de df_rj: {df_rj.shape}")
print (f"Número de linhas e colunas de df_sp: {df_sp.shape}")

Número de linhas e colunas de df_rj: (160, 24)
Número de linhas e colunas de df_sp: (296, 24)


Converter "faturamento" de df_rj para o tipo int, visto que não utiliza as casas decimais.

In [107]:
df_rj["faturamento"] = df_rj["faturamento"].astype(int).copy()
df_rj.head(2)

/var/folders/z7/spd20rg13kl9j0xgnkbzdrf40000gn/T/ipykernel_8631/3496868274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rj["faturamento"] = df_rj["faturamento"].astype(int).copy()


,codigo,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
0,3304557060,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501,932515,Médio
1,3304557138,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,588833,Baixo


df_rj será dividida em três partes, para realizar as análises 1 e 2 (problemas supervisionados de regressão e classificação), sendo:
- Dados de treinamento (train): utilizados para treinar os modelos de machine learning.
- Dados de validação (val): utilizados para validar a performance dos modelos, e para tunagem de hiperparâmetros.
- Dados de teste (test): utilizados para avaliação final da capacidade de generalização dos modelos, simulando dados de produção.

Para a análise 3 (problema não supervisionado de clusterização), não haverá separação entre treino, validação e teste, dado que a tarefa é agrupar os dados em 4 grupos, de acordo com as variáveis disponíveis no dataset.

df_sp será guardado, e utilizado como insumo dos modelos em sua versão final, a fim de realizar as análises 1, 2 e 3 (bairros de São Paulo).

## Estatística Descritiva

Realizada em cimda com 2 propósitos:
- Obter maior conhecimento de negócio a partir do comportamento dos dados.
- Detectar eventuais erros e sanar dúvidas sobre os dados, como faturamento negativo, por exemplo.

PS: Será realizada no ciclo 2.

# Análise 1 - Estimativa de faturamento em SP

Análise 1 - Estimar o faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão).

## Divisão entre Treino e Teste

Como a variável alvo da análise 1 (regressão) é "faturamento", e a da análise 2 (classificação) é "potencial", para cada análise, serão realizadas diferentes quebras entre treino, validação e teste.

Divisão entre features e variável alvo:

In [165]:
y1 = df_rj.faturamento
y1.head(2)

0    932515
1    588833
Name: faturamento, dtype: int64

In [166]:
X1 = df_rj.drop(["codigo","faturamento"], axis=1)
X1.head(2)

,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,potencial
0,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501,Médio
1,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,Baixo


Divisão entre datasets de treino e teste, separando 15% dos registros para teste.

In [167]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0, test_size=0.15)

In [168]:
print(X1_train.shape)
print(X1_test.shape)
X1_train.head(2)

(136, 22)
(24, 22)


,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,potencial
37,Cordovil,Rio de Janeiro,RJ,46478,5729,2893,3768,3850,7146,9369,5598,8125,0,0,1047,2433,4599,4053,723,2601,1480,Médio
33,Coelho Neto,Rio de Janeiro,RJ,33338,3923,1966,2517,2280,4796,7294,4280,6282,0,0,488,2295,3573,2820,470,2044,1425,Baixo


Divisão entre datasets de treino e validação (a partir do treino), separando 15% dos registros para validação.

In [174]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, random_state=0, test_size=0.15)

In [176]:
print(X1_train.shape)
print(X1_val.shape)
X1_train.head(2)

(115, 22)
(21, 22)


,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,potencial
63,Higienópolis,Rio de Janeiro,RJ,16177,1516,748,1016,1174,2488,3497,2327,3411,0,141,857,1464,1740,1030,87,432,2336,Médio
105,Pedra De Guaratiba,Rio de Janeiro,RJ,9666,1139,535,715,690,1415,2068,1284,1820,0,0,231,662,876,783,150,532,1522,Baixo


## Implementação de Modelo Baseline

Implementar KNN regressor:    -> descrever o algoritmo..

Manter apenas features numéricas, para respeitar a premissa do KNN de que a proximidade entre duas evidências de um fenômeno observado pode ser representada por uma medida de distância.

In [179]:
X1_train = X1_train.select_dtypes(include=['int64', 'float64'])
X1_val = X1_val.select_dtypes(include=['int64', 'float64'])
X1_train.head(2)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e
63,16177,1516,748,1016,1174,2488,3497,2327,3411,0,141,857,1464,1740,1030,87,432
105,9666,1139,535,715,690,1415,2068,1284,1820,0,0,231,662,876,783,150,532


Instanciar um KNN Regressor, treinar com os dados de treino, e realizar predições contra os dados de validação.

In [180]:
knn_reg = KNeighborsRegressor(n_neighbors=3)
knn_reg.fit(X1_train, y1_train)
knn_reg_yhat = knn_reg.predict(X1_val)

A métrica de performance será a R squared, ou coeficiente de determinação.
Ele é uma medita estatística de quão próximos os dados estão da linha de regressão ajustada.
Varia entre entre 0 e 100%, sendo a porcentagem da variação da variável resposta que é explicada por um modelo linear.

In [328]:
print(f"R^2 com dados de Validação: {round(knn_reg.score(X1_val, y1_val) ,3 )*100 } %")

R^2 com dados de Validação: 57.099999999999994 %


# Análise 2 - Classificação do potencial em SP

Análise 2 - Classificar o potencial dos bairros de São Paulo em alto, médio ou baixo (classificação multiclasse).

## Divisão entre Treino e Teste

Divisão entre features e variável alvo.

In [247]:
y2 = df_rj.potencial
y2.head(2)

0    Médio
1    Baixo
Name: potencial, dtype: object

In [248]:
X2 = df_rj.drop(["codigo","potencial"], axis=1)
X2.head(2)

,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento
0,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501,932515
1,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,588833


Avaliar balanceamento da var resposta.

In [249]:
y2.value_counts(normalize=True)*100

Baixo    38.75
Médio    31.25
Alto     30.00
Name: potencial, dtype: float64

Divisão entre datasets de treino e teste, separando 15% dos registros para teste.

Parâmetro stratify para manter a proportção da variável parecida no treino e teste.

In [250]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=0, test_size=0.15, stratify=y2)

In [251]:
print(X2_train.shape)
print(X2_test.shape)
X2_train.head(2)

(136, 22)
(24, 22)


,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento
4,Anchieta,Rio de Janeiro,RJ,57222,7677,3774,4892,4600,8660,12272,7157,8190,0,0,1089,2821,5110,5422,1073,3261,1380,553020
106,Penha,Rio de Janeiro,RJ,80900,9269,4616,7545,8520,12940,15618,9160,13232,0,239,2129,4629,7639,6086,1082,3563,1682,593191


Divisão entre datasets de treino e validação (a partir do treino), separando 15% dos registros para validação.

In [257]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_train, y2_train, random_state=0, test_size=0.15, stratify=y2_train)

In [259]:
print(X2_train.shape)
print(X2_val.shape)
X2_train.head(2)

(97, 22)
(18, 22)


,nome,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento
82,Leblon,Rio de Janeiro,RJ,47342,3134,1380,2131,2390,6177,9519,7181,15430,5423,3532,5342,1863,1890,917,217,1595,14738,2119774
6,Anil,Rio de Janeiro,RJ,24855,2427,1227,1777,1804,3730,5573,3556,4761,355,759,1771,1863,1902,1155,326,424,NaN,1092081


## Implementação de Modelo Baseline

Implementar KNN classifier:    -> descrever o algoritmo..

Manter apenas features numéricas, para respeitar a premissa do KNN de que a proximidade entre duas evidências de um fenômeno observado pode ser representada por uma medida de distância.

In [260]:
X2_train = X2_train.select_dtypes(include=['int64', 'float64'])
X2_val = X2_val.select_dtypes(include=['int64', 'float64'])
X2_train.head(2)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,faturamento
82,47342,3134,1380,2131,2390,6177,9519,7181,15430,5423,3532,5342,1863,1890,917,217,1595,2119774
6,24855,2427,1227,1777,1804,3730,5573,3556,4761,355,759,1771,1863,1902,1155,326,424,1092081


In [271]:
X2_val.head()

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,faturamento
128,42081,4341,2130,2887,3499,7584,9814,5201,6625,150,453,2290,2790,4882,3508,615,1565,703465
99,3270,369,158,229,195,389,566,492,872,0,10,115,244,380,329,65,108,854818
118,84538,10102,4229,5804,6423,14960,20404,10651,11965,2815,4307,7544,4011,4747,3339,800,2484,1596252
93,51234,3371,1719,2751,3246,6801,9850,8170,15326,510,1873,5463,5089,3911,1570,131,1356,1626856
140,30403,4008,1783,2398,2442,4930,6505,3601,4736,0,138,1081,1651,2673,2467,436,1962,752629


Instanciar um KNN Classifier, treinar com os dados de treino, e realizar predições contra os dados de validação.

In [272]:
knn_cla = KNeighborsClassifier(n_neighbors=3)
knn_cla.fit(X2_train, y2_train)
knn_cla_yhat = knn_cla.predict(X2_val)

In [274]:
knn_cla_yhat[:3]

array(['Alto', 'Baixo', 'Alto'], dtype=object)

A métrica de performance será a acurácia média, podendo ser utilizada aqui porque o dataset é balanceado. 
A acurácia mede quantas predições o modelo acertou, de todas as previsões realizadas.

In [322]:
print(f"Acurácia com dados de Validação: {round(knn_cla.score(X2_val, y2_val) ,3 )*100 } %")
#Here, the score is 0.83, which indicates a relatively good model fit.

Acurácia com dados de Validação: 77.8 %


# Análise 3 - Segmentação de perfis de aderência em SP

Análise 3 - Segmentar os bairros de São Paulo de acordo com o perfil de renda e idade, identificando os com maior aderência ao público alvo.